# XGBoost

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datasets/melb_data.csv")

# Нужные подмножества характеристик
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Целевые значения
y = data.Price

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

XGBoost расшифровывается как extreme gradient boosting, что представляет собой реализацию gradient boosting с несколькими дополнительными функциями, ориентированными на производительность и скорость. (В Scikit-learn есть другая версия градиентного бустинга, но у XGBoost есть некоторые технические преимущества)

In [3]:
from xgboost import XGBRegressor

In [4]:
my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

#### Прогноз и оценка модели

In [5]:
from sklearn.metrics import mean_absolute_error

In [6]:
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 245895.5626472754


#### Настройка параметров
В XGBoost есть несколько параметров, которые могут существенно повлиять на точность и скорость обучения. Первые параметры, которые вы должны понимать:

n_estimators указывает, сколько раз нужно пройти цикл моделирования, описанный выше. Это равно количеству моделей, которые мы включаем в ансамбль.

Слишком низкое значение приводит к недостаточной подгонке, что приводит к неточным прогнозам как по данным обучения, так и по тестовым данным.
Слишком высокое значение приводит к переобучению, что приводит к точным прогнозам по данным обучения, но неточным прогнозам по данным тестирования (это то, что нас волнует).
Типичные значения варьируются в пределах 100-1000, хотя это во многом зависит от параметра learning_rate

In [7]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 254843.9620455173


early_stopping_rounds предлагает способ автоматического поиска идеального значения для n_estimators. Ранняя остановка приводит к прекращению итерации модели, когда оценка проверки перестает улучшаться, даже если мы не достигли жесткой остановки для n_estimators. Разумно установить высокое значение для n_estimators, а затем использовать early_stopping_rounds, чтобы найти оптимальное время для остановки итерации.

Поскольку случайность иногда приводит к тому, что в одном раунде результаты проверки не улучшаются, вам нужно указать число, определяющее, сколько раундов прямого ухудшения нужно разрешить, прежде чем останавливаться. Установка early_stopping_rounds= 5 - разумный выбор. В этом случае мы останавливаемся после 5 раундов ухудшения результатов проверки подряд.

При использовании early_stopping_rounds вам также необходимо выделить некоторые данные для расчета баллов проверки - это делается путем установки параметра eval_set

In [8]:
my_model = XGBRegressor(n_estimators=500, early_stopping_rounds=5)
my_model.fit(X_train, y_train,  
             eval_set=[(X_valid, y_valid)],
             verbose=False)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 246034.34377761415


learning_rate

В целом, небольшая скорость обучения и большое количество оценок позволят получить более точные модели XGBoost, хотя также потребуется больше времени для обучения модели, поскольку она выполняет больше итераций в цикле. По умолчанию XGBoost устанавливает learning_rate=0.1

In [9]:
my_model = XGBRegressor(n_estimators=1000, early_stopping_rounds=5, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 244927.30751564802


n_jobs

На больших наборах данных, где учитывается время выполнения, вы можете использовать параллелизм для более быстрого построения ваших моделей. Обычно параметр n_jobs устанавливается равным количеству ядер на вашем компьютере. На небольших наборах данных это не поможет. Это полезно в больших наборах данных, где в противном случае вы бы потратили много времени на ожидание команды fit

In [10]:
my_model = XGBRegressor(
    n_estimators=1000, 
    early_stopping_rounds=5, 
    learning_rate=0.05, 
    n_jobs=4,
)

my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 244927.30751564802
